__Homework 3__

The goal of this homework is to investigate how minor changes in the prompting of an LLM can have a large effect on how well the model performs. We will use the MMLU dataset, which is a common benchmark for evaluating LLMs.

We will test and compare two methods for LLM prompting, namely Harness and Helm, which are explained in detail here: https://huggingface.co/blog/open-llm-leaderboard-mmlu

Most of the code is already implemented for you. Your task is to create two functions that return the prompts according to the helm and harness methods. Note that we do 5-shot prompting, so each input prompt to the LLM should have 5 questions with answers from the train set, and 1 question without answer from the test set.


In [3]:
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes
!pip install transformers datasets torch


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-nufe_zuh/unsloth_66ccf645b74b4283ab8a465139b0fbd6
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-nufe_zuh/unsloth_66ccf645b74b4283ab8a465139b0fbd6
  Resolved https://github.com/unslothai/unsloth.git to commit 5b10dbd497315804b6de40ca25d7541c3b460924
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.6/146.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.9 MB/s eta 0:

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.6/222.6 MB 2.6 MB/s eta 0:00:00


In [4]:
import torch

if torch.cuda.is_available():
    print("NVIDIA GPU is available.")
    print("Device name:", torch.cuda.get_device_name(0))
elif hasattr(torch, "xpu") and torch.xpu.is_available():
    print("Intel GPU is available.")
    print("Device name:", torch.xpu.get_device_name(0))
else:
    print("Neither NVIDIA nor Intel GPU is available or detected by PyTorch.")

NVIDIA GPU is available.
Device name: Tesla T4


In [5]:
# Import necessary libraries
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.2.2+cu121 with CUDA 1201 (you have 2.6.0+cu124)
    Python  3.11.8 (you have 3.11.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


In [6]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
# I used Tesla T4 GPU thus Float16 is working
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/llama-3-8b-bnb-4bit", # [NEW] 15 Trillion token Llama-3
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

In [7]:
import datasets
datasets.builder.has_sufficient_disk_space = lambda needed_bytes, directory='.': True

In [9]:
# Load the MMLU dataset (replace "lukaemon/mmlu" with the actual name/path of the dataset if it changes)
dataset = load_dataset("lukaemon/mmlu", 'electrical_engineering') # we only take the electrical engineering category
train_samples = dataset["train"]
test_samples = dataset["test"]


The repository for lukaemon/mmlu contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/lukaemon/mmlu.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating test split:   0%|          | 0/145 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/16 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/5 [00:00<?, ? examples/s]

In [10]:
print(len(train_samples), len(test_samples))

5 145


In [23]:
train_samples[0]

{'input': 'In an SR latch built from NOR gates, which condition is not allowed',
 'A': 'S=0, R=0',
 'B': 'S=0, R=1',
 'C': 'S=1, R=0',
 'D': 'S=1, R=1',
 'target': 'D'}

In [25]:
test_samples[0]

{'input': 'A working diode must have',
 'A': 'High resistance when forward or reverse biased',
 'B': 'Low resistance when forward biased, while high resistance when reverse bias',
 'C': 'High resistance when forward biased, while low resistance when reverse bias',
 'D': 'Low resistance when forward or reverse biased',
 'target': 'B'}

In [26]:
# Prepare 2 functions for the 5-shot prompt for Helm and Harness (with answers) and 1 test question (without answers)
import random


def helm_prompt(train_samples, test_sample): #5 train_samples, 1 test_sample

    prompt = ""
    num_shots = 5
    shots = random.sample(list(train_samples), num_shots)
    for sample in shots:
        question = sample["input"]
        answer_letter = sample["target"]
        answer_text = sample[answer_letter]
        prompt += f"Q: {question}\nA: {answer_text}\n\n"

    test_question= test_sample["input"]
    prompt += f"Q: {test_question.strip()}\nA:"
    return prompt


def harness_prompt(train_samples, test_sample): #5 train_samples, 1 test_sample
    prompt = ""
    num_shots = 5
    shots = random.sample(list(train_samples), num_shots)

    for sample in shots:
        question = sample["input"]
        prompt += f"{question}\n"

        for option in ["A", "B", "C", "D"]:
            prompt += f"{option} {sample[option]}\n"
        prompt += f"Answer: {sample['target']}\n\n"

    test_question= test_sample["input"]
    prompt += f"{test_question.strip()}\n"
    for option in ["A", "B", "C", "D"]:
            prompt += f"{option} {test_sample[option]}\n"
    return prompt




In [27]:
def evaluate(train_samples, test_samples, mode = 'helm'):#mode can be harness or helm
    correct_count = 0

    for i in range(len(test_samples)):

        if mode == 'helm':
            input_prompt = helm_prompt(train_samples, test_samples[i])
        else:
            input_prompt = harness_prompt(train_samples, test_samples[i])

        inputs = tokenizer(input_prompt, return_tensors='pt')

        # Ensure tokens are on the same device as the model
        input_ids = inputs['input_ids'].to(model.device)

        outputs = model.generate(
            input_ids=input_ids,
            do_sample=False, # No sampling, only output the most probable token
            max_length=len(input_ids[0])+1, #Output length will be input length + 1. This extra token is the answer
            pad_token_id=tokenizer.eos_token_id,
        )

        # Decode the output tokens to get the final response
        generated_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract just the newly generated token after the input prompt
        final_answer = generated_response[-1]
        #print(i, final_answer, test_samples[i]['target'])
        if final_answer==test_samples[i]['target']:
            correct_count +=1

    return(correct_count/len(test_samples))

In [28]:
helm_acc = evaluate(train_samples, test_samples, 'helm')
harness_acc = evaluate(train_samples, test_samples, 'harness')

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [29]:
print('The accuracy using the Helm prompting is ', helm_acc)
print('The accuracy using the Harness prompting is ', harness_acc)

The accuracy using the Helm prompting is  0.11724137931034483
The accuracy using the Harness prompting is  0.06206896551724138
